In [41]:
iteration = 1

In [56]:
# Parameters
iteration = 4


In [57]:
config_ = {
    'input_dim': (5,5),
    'encoding_dim': 100,
    'input_size': (4,5),
    'num_layers': 1,
    'output_size': 1,
    'n_future': 1,
    'n_past': 5   
}

In [58]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
from sklearn.metrics import mean_absolute_percentage_error
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, LeakyReLU
from tensorflow.keras.callbacks import EarlyStopping
import keras_uncertainty
from keras_uncertainty.layers import StochasticDropout
from keras_uncertainty.models import StochasticRegressor
from keras_uncertainty.utils import regressor_calibration_error

In [59]:
AAAFF = pd.read_csv('raw_data/AAAFF.csv').fillna(method = 'ffill').dropna()
DTB3 = pd.read_csv('raw_data/DTB3.csv').fillna(method = 'ffill').dropna()
SnP_div = pd.read_excel('raw_data/S&P Dividend Yield 1963-2023.xlsx').rename(columns={'Name':'DATE'}).set_index('DATE').resample('d').fillna(method = 'ffill').dropna().reset_index()
SnP = pd.read_excel('raw_data/SP500Index_Retrieved_26_06_2022.xlsx').loc[5:]
SnP = SnP.rename(columns={SnP.columns[0]:'DATE',SnP.columns[1]:'SnP PRICE'}).set_index('DATE').resample('d').fillna(method = 'ffill').dropna().reset_index()
TED = pd.read_csv('raw_data/TEDRATE.csv').fillna(method = 'ffill').dropna()
for i, item in enumerate([TED, AAAFF, DTB3, SnP_div, SnP]):
    item['DATE'] = pd.to_datetime(item['DATE'])
    if i == 0:
        data = item.copy().set_index('DATE')
    else:
        data = data.join(item.set_index('DATE'), on='DATE', how='left')
data = data.replace('.', None).fillna(method = 'ffill').dropna()
data = data.astype(float)

#MinMax Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
scaled_data = pd.DataFrame(scaled_data, index=data.index, columns=data.columns)
unscaler = MinMaxScaler()
unscaler = unscaler.fit(data[['SnP PRICE']])

/var/folders/5_/7k75vhv540d_lhcr6xp_xrzw0000gn/T/ipykernel_69716/4287731575.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  AAAFF = pd.read_csv('raw_data/AAAFF.csv').fillna(method = 'ffill').dropna()
/var/folders/5_/7k75vhv540d_lhcr6xp_xrzw0000gn/T/ipykernel_69716/4287731575.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  DTB3 = pd.read_csv('raw_data/DTB3.csv').fillna(method = 'ffill').dropna()
/var/folders/5_/7k75vhv540d_lhcr6xp_xrzw0000gn/T/ipykernel_69716/4287731575.py:3: FutureWarning: DatetimeIndexResampler.fillna is deprecated and will be removed in a future version. Use obj.ffill(), obj.bfill(), or obj.nearest() instead.
  SnP_div = pd.read_excel('raw_data/S&P Dividend Yield 1963-2023.xlsx').rename(columns={'Name':'DATE'}).set_index('DATE').resample('d').fillna(method = 'ffill').dropna().re

In [60]:
train_data_1 = scaled_data.loc["1971-02-05":"1973-03-07"]
val_data_1 = scaled_data.loc["1973-06-12":"1974-03-26"]
test_data_1 = scaled_data.loc["1974-03-28":"1976-02-02"]

train_data_2 = scaled_data.loc["1975-08-19":"1979-05-18"]
val_data_2 = scaled_data.loc["1979-11-06":"1980-02-25"]
test_data_2 = scaled_data.loc["1980-02-26":"1980-11-06"]

train_data_3 = scaled_data.loc["1980-07-24":"1981-01-13"]
val_data_3 = scaled_data.loc["1981-02-04":"1981-11-23"]
test_data_3 = scaled_data.loc["1981-11-24":"1983-10-07"]

train_data_4 = scaled_data.loc["1983-08-02":"1989-07-19"]
val_data_4 = scaled_data.loc["1990-04-19":"1990-09-12"]
test_data_4 = scaled_data.loc["1990-09-13":"1991-08-19"]
dataset_4 = {'train': train_data_4, 'val': val_data_4, 'test': test_data_4}

train_data_5 = scaled_data.loc["1992-02-24":"1999-12-22"]
val_data_5 = scaled_data.loc["2000-12-15":"2001-05-10"]
test_data_5 = scaled_data.loc["2001-05-11":"2002-04-24"]
dataset_5 = {'train': train_data_5, 'val': val_data_5, 'test': test_data_5}

train_data_6 = scaled_data.loc["2002-05-30":"2006-11-24"]
val_data_6 = scaled_data.loc["2007-06-22":"2008-05-14"]
test_data_6 = scaled_data.loc["2008-05-15":"2010-06-16"]
dataset_6 = {'train': train_data_6, 'val': val_data_6, 'test': test_data_6}

train_data_7 = scaled_data.loc["2010-06-23":"2018-12-19"]
val_data_7 = scaled_data.loc["2020-01-14":"2020-02-20"]
test_data_7 = scaled_data.loc["2020-02-21":"2020-05-13"]
dataset_7 = {'train': train_data_7, 'val': val_data_7, 'test': test_data_7}

# print(f"Data 1: {len(train_data_1)}")
# print(f"Data 2: {len(train_data_2)}")
# print(f"Data 3: {len(train_data_3)}")
print(f"Data 4: {len(train_data_4)}")
print(f"Data 5: {len(train_data_5)}")
print(f"Data 6: {len(train_data_6)}")
print(f"Data 7: {len(train_data_7)}")


Data 4: 925
Data 5: 2043
Data 6: 1172
Data 7: 2216


In [61]:
match(iteration):
    case 1:
        selected_dataset = dataset_4
    case 2:
        selected_dataset = dataset_5
    case 3: 
        selected_dataset = dataset_6
    case 4: 
        selected_dataset = dataset_7


In [63]:
# Shaping Data
def shape_data(data, n_future, n_past):
    X = []
    y = []
    data = data.to_numpy()
    for i in range(n_past, data.shape[0] - n_future +1): 
        X.append(data[i - n_past:i, 0:data.shape[1]])
        y.append(data[i + n_future - 1:i + n_future,data.shape[1]-1])

    X, y = np.array(X), np.array(y)

    print('TrainX shape = {}.'.format(X.shape))
    print('TrainY shape = {}.'.format(y.shape))
    return X, y

n_future = config_['n_future']
n_past = config_['n_past']
trainX, trainY = shape_data(selected_dataset['train'], n_future, n_past)
valX, valY = shape_data(selected_dataset['val'], n_future, n_past)
testX, testY = shape_data(selected_dataset['test'], n_future, n_past)


TrainX shape = (2211, 5, 5).
TrainY shape = (2211, 1).
TrainX shape = (23, 5, 5).
TrainY shape = (23, 1).
TrainX shape = (54, 5, 5).
TrainY shape = (54, 1).


In [64]:
# how well the predicted uncertainty (in this case, represented by y_std) matches the actual accuracy of the predictions.
from scipy.stats import norm
import numpy as np

dropout_probability = 0.2

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
input_shape = config_['input_dim']
encoding_dim = config_['encoding_dim']

# predictor
input_layer_1 = Input(shape=input_shape)
encoder_output = Dense(encoding_dim, activation='relu', name="encoder")(input_layer_1)
flattened_output = Reshape((1,-1))(encoder_output)
dense_layer = Dense(100)(flattened_output)
predicted = Dense(1)(dense_layer)

predictor_model = Model(inputs=input_layer_1, outputs=predicted)
predictor_model.compile(optimizer='adam', loss='mse')
print(predictor_model.summary())

history_pm = predictor_model.fit(trainX, trainY, epochs=100, batch_size=32, validation_data=(valX, valY), verbose=1, callbacks=[early_stopping])
encoder_layer = predictor_model.get_layer('encoder')

# mc dropout predictor 
input_layer_2 = Input(shape=input_shape)
encoded_data = encoder_layer(input_layer_2)
encoder_layer.trainable = False
dropped_data_1 = StochasticDropout(dropout_probability)(encoded_data)
flattened_output = Reshape((1,-1))(dropped_data_1)  
dense_output = Dense(1)(flattened_output)
predicted_2 = LeakyReLU(alpha=0.3)(dense_output)

mc_predictor_model = Model(inputs=input_layer_2, outputs=predicted_2)
mc_predictor_model.compile(optimizer='adam', loss='mse')

print(mc_predictor_model.summary())

history_mc_pm = mc_predictor_model.fit(trainX, trainY, epochs=100, batch_size=32, validation_data=(valX, valY), verbose=1, callbacks=[early_stopping])
mc_model = StochasticRegressor(mc_predictor_model)

#autoencoder
input_layer = Input(shape=input_shape)
encoded_data = encoder_layer(input_layer)
encoder_layer.trainable = False
decoder_output = Dense(5, activation='linear')(encoded_data)
output_tensor = Reshape(input_shape)(decoder_output)
autoencoder = Model(inputs=input_layer, outputs=output_tensor)
autoencoder.compile(optimizer='adam', loss='mse')
print(autoencoder.summary())

history_ae = autoencoder.fit(trainX, trainX, epochs=100, batch_size=32, validation_data=(valX, valX), verbose=1, callbacks=[early_stopping])

2024-06-07 22:35:41.985027: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-07 22:35:41.985078: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-07 22:35:41.985091: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-07 22:35:41.985341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-07 22:35:41.985672: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5, 5)]            0         
                                                                 
 encoder (Dense)             (None, 5, 100)            600       
                                                                 
 reshape (Reshape)           (None, 1, 500)            0         
                                                                 
 dense (Dense)               (None, 1, 100)            50100     
                                                                 
 dense_1 (Dense)             (None, 1, 1)              101       
                                                                 
Total params: 50801 (198.44 KB)
Trainable params: 50801 (198.44 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/100


2024-06-07 22:35:43.009412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


70/70 [==============================] - 5s 36ms/step - loss: 0.0046 - val_loss: 0.0016
Epoch 2/100
70/70 [==============================] - 1s 10ms/step - loss: 6.6152e-05 - val_loss: 9.0130e-04
Epoch 3/100
70/70 [==============================] - 1s 10ms/step - loss: 4.2209e-05 - val_loss: 8.7165e-04
Epoch 4/100
70/70 [==============================] - 1s 10ms/step - loss: 4.9097e-05 - val_loss: 7.5865e-04
Epoch 5/100
70/70 [==============================] - 1s 10ms/step - loss: 3.9776e-05 - val_loss: 4.7165e-04
Epoch 6/100
70/70 [==============================] - 1s 10ms/step - loss: 4.4894e-05 - val_loss: 2.3745e-04
Epoch 7/100
70/70 [==============================] - 1s 9ms/step - loss: 4.0724e-05 - val_loss: 5.2563e-04
Epoch 8/100
70/70 [==============================] - 1s 9ms/step - loss: 2.9558e-05 - val_loss: 5.8102e-04
Epoch 9/100
70/70 [==============================] - 1s 9ms/step - loss: 6.1344e-05 - val_loss: 3.9235e-04
Epoch 10/100
70/70 [==============================]

In [65]:
scaler = unscaler

In [67]:
#train
un_predictions = predictor_model.predict(trainX).flatten()
predictions = scaler.inverse_transform(un_predictions.reshape(-1, 1))

ae_output = autoencoder.predict(trainX)
error_dict = {}
for num, column in enumerate(data.columns):
    error_dict[column+' error'] = []
    for i in range(trainX.shape[0]):
        error_dict[column+' error'].append(mean_absolute_percentage_error(ae_output[i,:,num], trainX[i,:,num]))
    error_dict[column+' error'] = np.array(error_dict[column+' error'])

errors = []

for i in range(trainX.shape[0]):
    errors.append(mean_absolute_percentage_error(ae_output[i], trainX[i]))
error_dict["errors"] = np.array(errors)
#shape is (iteration,day,column)

gaussian_score = np.array([])
calib_err = np.array([])
for i in range(len(trainX)):
    y_pred_mean, y_pred_std = mc_model.predict(trainX[i:i+1,:,:], num_samples=100)
    gaussian_score = np.append(gaussian_score, y_pred_std)

rl_train = pd.DataFrame(scaler.inverse_transform(selected_dataset['train'].iloc[5:].copy()), index=selected_dataset['train'].iloc[5:].index, columns=selected_dataset['train'].iloc[5:].columns)

for error_type in error_dict.keys():
    rl_train[error_type] = error_dict[error_type]

rl_train['predictions'] = predictions
rl_train['gauss_score'] = gaussian_score

#val
un_predictions = predictor_model.predict(valX).flatten()
predictions = scaler.inverse_transform(un_predictions.reshape(-1, 1))

ae_output = autoencoder.predict(valX)
error_dict = {}
for num, column in enumerate(data.columns):
    error_dict[column+' error'] = []
    for i in range(valX.shape[0]):
        error_dict[column+' error'].append(mean_absolute_percentage_error(ae_output[i,:,num], valX[i,:,num]))
    error_dict[column+' error'] = np.array(error_dict[column+' error'])

errors = []

for i in range(valX.shape[0]):
    errors.append(mean_absolute_percentage_error(ae_output[i], valX[i]))
error_dict["errors"] = np.array(errors)
#shape is (iteration,day,column)

gaussian_score = np.array([])
calib_err = np.array([])
for i in range(len(valX)):
    y_pred_mean, y_pred_std = mc_model.predict(valX[i:i+1,:,:], num_samples=100)
    gaussian_score = np.append(gaussian_score, y_pred_std)

rl_val = pd.DataFrame(scaler.inverse_transform(selected_dataset['val'].iloc[5:].copy()), index=selected_dataset['val'].iloc[5:].index, columns=selected_dataset['val'].iloc[5:].columns)

for error_type in error_dict.keys():
    rl_val[error_type] = error_dict[error_type]

rl_val['predictions'] = predictions
rl_val['gauss_score'] = gaussian_score

#test
un_predictions = predictor_model.predict(testX).flatten()
predictions = scaler.inverse_transform(un_predictions.reshape(-1, 1))

ae_output = autoencoder.predict(testX)
error_dict = {}
for num, column in enumerate(data.columns):
    error_dict[column+' error'] = []
    for i in range(testX.shape[0]):
        error_dict[column+' error'].append(mean_absolute_percentage_error(ae_output[i,:,num], testX[i,:,num]))
    error_dict[column+' error'] = np.array(error_dict[column+' error'])

errors = []

for i in range(testX.shape[0]):
    errors.append(mean_absolute_percentage_error(ae_output[i], testX[i]))
error_dict["errors"] = np.array(errors)
#shape is (iteration,day,column)

gaussian_score = np.array([])
calib_err = np.array([])
for i in range(len(testX)):
    y_pred_mean, y_pred_std = mc_model.predict(testX[i:i+1,:,:], num_samples=100)
    gaussian_score = np.append(gaussian_score, y_pred_std)

rl_test = pd.DataFrame(scaler.inverse_transform(selected_dataset['test'].iloc[5:].copy()), index=selected_dataset['test'].iloc[5:].index, columns=selected_dataset['test'].iloc[5:].columns)

for error_type in error_dict.keys():
    rl_test[error_type] = error_dict[error_type]

rl_test['predictions'] = predictions
rl_test['gauss_score'] = gaussian_score


 9/70 [==>...........................] - ETA: 0s

2/2 [==============================] - 0s 169ms/step


In [68]:
import numpy as np
from stockstats import StockDataFrame as Sdf

"""Taken from finRL"""

class FeatureEngineer:
    """Provides methods for preprocessing the stock price data

    Attributes
    ----------
        use_technical_indicator : boolean
            we technical indicator or not
        tech_indicator_list : list
            a list of technical indicator names (modified from neofinrl_config.py)
        use_turbulence : boolean
            use turbulence index or not
        user_defined_feature:boolean
            use user defined features or not

    Methods
    -------
    preprocess_data()
        main method to do the feature engineering

    """

    def __init__(
        self,
        use_technical_indicator=True,
        tech_indicator_list = ["macd", "boll","boll_ub","boll_lb","rsi_30","cci_30","dx_30","close_30_sma","close_60_sma"],
        use_vix=False,
        use_turbulence=False,
        user_defined_feature=False,
    ):
        self.use_technical_indicator = use_technical_indicator
        self.tech_indicator_list = tech_indicator_list
        self.use_vix = use_vix
        self.use_turbulence = use_turbulence
        self.user_defined_feature = user_defined_feature

    def preprocess_data(self, df):
        """main method to do the feature engineering
        @:param config: source dataframe
        @:return: a DataMatrices object
        """
        # clean data
        df = self.clean_data(df)

        # add technical indicators using stockstats
        if self.use_technical_indicator:
            df = self.add_technical_indicator(df)
            print("Successfully added technical indicators")

        # add vix for multiple stock
        if self.use_vix:
            df = self.add_vix(df)
            print("Successfully added vix")

        # add turbulence index for multiple stock
        if self.use_turbulence:
            df = self.add_turbulence(df)
            print("Successfully added turbulence index")

        # add user defined feature
        if self.user_defined_feature:
            df = self.add_user_defined_feature(df)
            print("Successfully added user defined features")

        # fill the missing values at the beginning and the end
        df = df.ffill().bfill()
        return df

    def clean_data(self, data):
        """
        clean the raw data
        deal with missing values
        reasons: stocks could be delisted, not incorporated at the time step
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        df = df.sort_values(["date", "tic"], ignore_index=True)
        df.index = df.date.factorize()[0]
        merged_closes = df.pivot_table(index="date", columns="tic", values="close")
        merged_closes = merged_closes.dropna(axis=1)
        tics = merged_closes.columns
        df = df[df.tic.isin(tics)]
        # df = data.copy()
        # list_ticker = df["tic"].unique().tolist()
        # only apply to daily level data, need to fix for minute level
        # list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
        # combination = list(itertools.product(list_date,list_ticker))

        # df_full = pd.DataFrame(combination,columns=["date","tic"]).merge(df,on=["date","tic"],how="left")
        # df_full = df_full[df_full['date'].isin(df['date'])]
        # df_full = df_full.sort_values(['date','tic'])
        # df_full = df_full.fillna(0)
        return df

    def add_technical_indicator(self, data):
        """
        calculate technical indicators
        use stockstats package to add technical inidactors
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        df = df.sort_values(by=["tic", "date"])
        stock = Sdf.retype(df.copy())
        unique_ticker = stock.tic.unique()

        for indicator in self.tech_indicator_list:
            indicator_df = pd.DataFrame()
            for i in range(len(unique_ticker)):
                try:
                    temp_indicator = stock[stock.tic == unique_ticker[i]][indicator]
                    temp_indicator = pd.DataFrame(temp_indicator)
                    temp_indicator["tic"] = unique_ticker[i]
                    temp_indicator["date"] = df[df.tic == unique_ticker[i]][
                        "date"
                    ].to_list()
                    # indicator_df = indicator_df.append(
                    #     temp_indicator, ignore_index=True
                    # )
                    indicator_df = pd.concat(
                        [indicator_df, temp_indicator], axis=0, ignore_index=True
                    )
                except Exception as e:
                    print(e)
            df = df.merge(
                indicator_df[["tic", "date", indicator]], on=["tic", "date"], how="left"
            )
        df = df.sort_values(by=["date", "tic"])
        return df
        # df = data.set_index(['date','tic']).sort_index()
        # df = df.join(df.groupby(level=0, group_keys=False).apply(lambda x, y: Sdf.retype(x)[y], y=self.tech_indicator_list))
        # return df.reset_index()

    def add_user_defined_feature(self, data):
        """
         add user defined features
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        df["daily_return"] = df.close.pct_change(1)
        # df['return_lag_1']=df.close.pct_change(2)
        # df['return_lag_2']=df.close.pct_change(3)
        # df['return_lag_3']=df.close.pct_change(4)
        # df['return_lag_4']=df.close.pct_change(5)
        return df

    def add_vix(self, data):
        """
        add vix from yahoo finance
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        df_vix = YahooDownloader(
            start_date=df.date.min(), end_date=df.date.max(), ticker_list=["^VIX"]
        ).fetch_data()
        vix = df_vix[["date", "close"]]
        vix.columns = ["date", "vix"]

        df = df.merge(vix, on="date")
        df = df.sort_values(["date", "tic"]).reset_index(drop=True)
        return df

    def add_turbulence(self, data):
        """
        add turbulence index from a precalcualted dataframe
        :param data: (df) pandas dataframe
        :return: (df) pandas dataframe
        """
        df = data.copy()
        turbulence_index = self.calculate_turbulence(df)
        df = df.merge(turbulence_index, on="date")
        df = df.sort_values(["date", "tic"]).reset_index(drop=True)
        return df

    def calculate_turbulence(self, data):
        """calculate turbulence index based on dow 30"""
        # can add other market assets
        df = data.copy()
        df_price_pivot = df.pivot(index="date", columns="tic", values="close")
        # use returns to calculate turbulence
        df_price_pivot = df_price_pivot.pct_change()

        unique_date = df.date.unique()
        # start after a year
        start = 252
        turbulence_index = [0] * start
        # turbulence_index = [0]
        count = 0
        for i in range(start, len(unique_date)):
            current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
            # use one year rolling window to calcualte covariance
            hist_price = df_price_pivot[
                (df_price_pivot.index < unique_date[i])
                & (df_price_pivot.index >= unique_date[i - 252])
            ]
            # Drop tickers which has number missing values more than the "oldest" ticker
            filtered_hist_price = hist_price.iloc[
                hist_price.isna().sum().min() :
            ].dropna(axis=1)

            cov_temp = filtered_hist_price.cov()
            current_temp = current_price[[x for x in filtered_hist_price]] - np.mean(
                filtered_hist_price, axis=0
            )
            # cov_temp = hist_price.cov()
            # current_temp=(current_price - np.mean(hist_price,axis=0))

            temp = current_temp.values.dot(np.linalg.pinv(cov_temp)).dot(
                current_temp.values.T
            )
            if temp > 0:
                count += 1
                if count > 2:
                    turbulence_temp = temp[0][0]
                else:
                    # avoid large outlier because of the calculation just begins
                    turbulence_temp = 0
            else:
                turbulence_temp = 0
            turbulence_index.append(turbulence_temp)
        try:
            turbulence_index = pd.DataFrame(
                {"date": df_price_pivot.index, "turbulence": turbulence_index}
            )
        except ValueError:
            raise Exception("Turbulence information could not be added.")
        return turbulence_index

"""Contains methods and classes to collect data from
Yahoo Finance API
"""

from __future__ import annotations

import pandas as pd
import yfinance as yf


class YahooDownloader:
    """Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from neofinrl_config.py)
        end_date : str
            end date of the data (modified from neofinrl_config.py)
        ticker_list : list
            a list of stock tickers (modified from neofinrl_config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API

    """

    def __init__(self, start_date: str, end_date: str, ticker_list: list):
        self.start_date = start_date
        self.end_date = end_date
        self.ticker_list = ticker_list

    def fetch_data(self, proxy=None) -> pd.DataFrame:
        """Fetches data from Yahoo API
        Parameters
        ----------
        Returns
        -------
        `pd.DataFrame`
            7 columns: A date, open, high, low, close, volume and tick symbol
            for the specified stock ticker
        """
        # Download and save the data in a pandas DataFrame:
        data_df = pd.DataFrame()
        num_failures = 0
        for tic in self.ticker_list:
            temp_df = yf.download(
                tic, start=self.start_date, end=self.end_date, proxy=proxy
            )
            temp_df["tic"] = tic
            if len(temp_df) > 0:
                # data_df = data_df.append(temp_df)
                data_df = pd.concat([data_df, temp_df], axis=0)
            else:
                num_failures += 1
        if num_failures == len(self.ticker_list):
            raise ValueError("no data is fetched.")
        # reset the index, we want to use numbers as index instead of dates
        data_df = data_df.reset_index()
        try:
            # convert the column names to standardized names
            data_df.columns = [
                "date",
                "open",
                "high",
                "low",
                "close",
                "adjcp",
                "volume",
                "tic",
            ]
            # use adjusted close price instead of close price
            data_df["close"] = data_df["adjcp"]
            # drop the adjusted close price column
            data_df = data_df.drop(labels="adjcp", axis=1)
        except NotImplementedError:
            print("the features are not supported currently")
        # create day of the week column (monday = 0)
        data_df["day"] = data_df["date"].dt.dayofweek
        # convert date to standard string format, easy to filter
        data_df["date"] = data_df.date.apply(lambda x: x.strftime("%Y-%m-%d"))
        # drop missing data
        data_df = data_df.dropna()
        data_df = data_df.reset_index(drop=True)
        print("Shape of DataFrame: ", data_df.shape)
        # print("Display DataFrame: ", data_df.head())

        data_df = data_df.sort_values(by=["date", "tic"]).reset_index(drop=True)

        return data_df

    def select_equal_rows_stock(self, df):
        df_check = df.tic.value_counts()
        df_check = pd.DataFrame(df_check).reset_index()
        df_check.columns = ["tic", "counts"]
        mean_df = df_check.counts.mean()
        equal_list = list(df.tic.value_counts() >= mean_df)
        names = df.tic.value_counts().index
        select_stocks_list = list(names[equal_list])
        df = df[df.tic.isin(select_stocks_list)]
        return df

def add_indicators(og_df):
    start_date = og_df.index[0]
    end_date = og_df.index[-1]
    downloader = YahooDownloader(start_date=start_date, end_date=end_date, ticker_list=['^SPX'])
    new_data = downloader.fetch_data()
    new_data['date'] = pd.to_datetime(new_data['date'])
    df = pd.merge(left = new_data, right = og_df.reset_index().drop(columns = 'SnP PRICE').rename(columns={"DATE":"date"}), on='date')

    fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = ["macd", "macds", "macdh", "atr","boll_ub","boll_lb","boll", "rsi"],
                     use_turbulence=False,
                     user_defined_feature = False)

    processed = fe.preprocess_data(df)
    processed = processed.copy()
    processed = processed.fillna(0)
    processed = processed.replace(np.inf,0)
    return processed

In [69]:
temp_array = ["rl_train", "rl_val", "rl_test"]
for i, items in enumerate([rl_train, rl_val, rl_test]):
    items = add_indicators(items)
    items.to_parquet(f'datasets/{temp_array[i]}_dataset{iteration+3}.parquet')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Shape of DataFrame:  (2133, 8)
Successfully added technical indicators
Shape of DataFrame:  (21, 8)
Successfully added technical indicators
Shape of DataFrame:  (52, 8)
Successfully added technical indicators


In [71]:
import papermill as pm
# parameters
iterations = [1,2,3]

for iteration in (iterations):
    print(f"iteration: {iteration}")
    print('========================')
    pm.execute_notebook('dataset_preparation.ipynb',
                        'dataset_preparation.ipynb',
                        parameters=dict(iteration = iteration)
                        )

iteration: 1


Executing:  50%|█████     | 7/14 [00:06<00:04,  1.50cell/s]2024-06-08 00:44:10.842785: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-08 00:44:10.842810: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-08 00:44:10.842820: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-08 00:44:10.842850: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-08 00:44:10.842871: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-06-08 00:44:11.787183: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer 

iteration: 2


Executing:  50%|█████     | 7/14 [00:06<00:04,  1.43cell/s]2024-06-08 01:51:41.935805: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-08 01:51:41.935827: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-08 01:51:41.935833: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-08 01:51:41.935873: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-08 01:51:41.935890: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-06-08 01:51:42.916171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer 

iteration: 3


Executing:  50%|█████     | 7/14 [00:05<00:04,  1.52cell/s]2024-06-08 03:58:47.252969: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-06-08 03:58:47.252995: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-06-08 03:58:47.253003: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-06-08 03:58:47.253064: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-08 03:58:47.253085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-06-08 03:58:48.235316: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer 